In [1]:
import os
import glob
import pandas as pd

In [2]:
pwd

'/Users/michaelpiacentino/Drive/git/data/smpd3/Sox9Counts'

In [3]:
# Add experiment date here to apply to dataframe
analysis_date = '20181110'

path = os.path.abspath('')+'/CSVs/'
full_df = pd.DataFrame()
list_ = []

# For loop to bring in files and concatenate them into a single dataframe
for file_ in glob.glob(path + "/*.csv"):
    df = pd.read_csv(file_)
    # Determine Image name from file name
    df['Image'] = os.path.splitext(os.path.basename(file_))[0]
    # Split values in Image name column
    (df['ExptDate'], df['Treatment'], df['Stains'], df['Embryo'], 
        df['Somites'], df['ImageMag'], df['Section']) = zip(*df['Image'].map(lambda x: x.split('_')))
    list_.append(df)

full_df = pd.concat(list_)
full_df.head()

,Slice,Count,Total Area,Average Size,%Area,Mean,IntDen,Image,ExptDate,Treatment,Stains,Embryo,Somites,ImageMag,Section
0,CntlSide,14,408.213,29.158,3.327,255.0,7435.305,20171016_SMPD3gRNA1DNA_Pax7Sox9_Emb7_8ss_20x_sec6,20171016,SMPD3gRNA1DNA,Pax7Sox9,Emb7,8ss,20x,sec6
1,ExptSide,10,379.923,37.992,3.045,255.0,9688.046,20171016_SMPD3gRNA1DNA_Pax7Sox9_Emb7_8ss_20x_sec6,20171016,SMPD3gRNA1DNA,Pax7Sox9,Emb7,8ss,20x,sec6
0,CntlSide,28,1117.407,39.907,5.579,255.0,10176.381,20171016_SMPD3gRNA1DNA_Pax7Sox9_Emb7_8ss_20x_sec5,20171016,SMPD3gRNA1DNA,Pax7Sox9,Emb7,8ss,20x,sec5
1,ExptSide,27,1104.679,40.914,5.479,255.0,10433.078,20171016_SMPD3gRNA1DNA_Pax7Sox9_Emb7_8ss_20x_sec5,20171016,SMPD3gRNA1DNA,Pax7Sox9,Emb7,8ss,20x,sec5
0,CntlSide,25,1085.922,43.437,5.434,255.0,11076.408,20171016_SMPD3gRNA1DNA_Pax7Sox9_Emb7_8ss_20x_sec4,20171016,SMPD3gRNA1DNA,Pax7Sox9,Emb7,8ss,20x,sec4


In [4]:
# Get a list of treatments
treatment_list = full_df.Treatment.unique()
treatment_list = treatment_list.tolist()

# Mean counts across sections
mean_sections = pd.DataFrame((full_df.groupby(['Treatment', 'Embryo', 'Slice', 'ExptDate'])['Count']).mean())

# Loop trough treatments, performing each analysis and exporting CSV file for each treatment
for i in treatment_list:
    # Slice dataframe to process only embryos with given treatment
    treatment = i
    df_treatment = pd.DataFrame(mean_sections.xs(treatment))

    # Extract means for Cntl and Expt counts
    counts_cntl = df_treatment.xs('CntlSide', level='Slice')['Count'] 
    counts_expt = df_treatment.xs('ExptSide', level='Slice')['Count']

    # Generate ratios as Expt/Cntl
    counts_ratios = pd.DataFrame(counts_expt / counts_cntl)
    counts_ratios.columns = ['Expt/Cntl Counts']

    # Normalize individual values to mean of control group
    norm_cntl = counts_cntl/(float(counts_cntl.mean()))
    norm_expt = counts_expt/(float(counts_cntl.mean()))

    # Combine processed values into single dataframe and output as csv file
    counts_cntl = pd.DataFrame(counts_cntl)
    counts_cntl.columns = ['Cntl Counts']
    counts_expt = pd.DataFrame(counts_expt)
    counts_expt.columns = ['Expt Counts']
    counts_ratios = pd.DataFrame(counts_ratios)
    counts_ratios.columns = ['Expt/Cntl Counts']
    norm_cntl = pd.DataFrame(norm_cntl)
    norm_cntl.columns = ['Norm Cntl Counts']
    norm_expt = pd.DataFrame(norm_expt)
    norm_expt.columns = ['Norm Expt Counts']
    results = (pd.concat([counts_cntl, counts_expt, counts_ratios, norm_cntl, norm_expt], axis=1, sort=True)).reset_index()
    results['ID'] = results.ExptDate.str.cat(results.Embryo)
    
    # Save out results at CSV file, update file name
    results.to_csv(analysis_date + '_' + treatment + '_Sox9CountResults.csv')